<a href="https://colab.research.google.com/github/BillyImperial/ProjetoGit/blob/main/Day_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile data.py
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def start_pipeline(dataf):
    return dataf.copy()

def get_product_counts(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'customer_id': pd.Series.nunique}) \
    .rename(columns={'customer_id': 'customer_id_count'}) \
    .set_index('product_id') \
    .sort_values('customer_id_count', ascending = False)

def get_avg_ratings(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'review_score': np.mean}) \
    .rename(columns={'review_score': 'review_score_avg'}) \
    .set_index('product_id') \
    .sort_values('review_score_avg', ascending = False)

def get_n_top_values(dataf, n):
  return dataf.head(n) \
    .index \
    .tolist()

top_product_ratings = df \
  .pipe(start_pipeline) \
  .pipe(get_avg_ratings) \
  .pipe(get_n_top_values, n=1500)

top_product_sellers = df \
  .pipe(start_pipeline) \
  .pipe(get_product_counts) \
  .pipe(get_n_top_values, n=10)

top_product_ratings[:10], top_product_sellers 

Overwriting data.py


In [ ]:
%%writefile item_based_recommender.py
import random

import numpy as np

from data import get_data, get_item_based_similarity_matrix


class ItemBasedRecommender:

  def __init__(self, data_path):
    self.data_path = data_path

  def setup(self):
    data = get_data(self.data_path)
    sim_matrix, available_items_index = get_item_based_similarity_matrix(data)
    self.available_items_index = available_items_index
    self.sim_matrix = sim_matrix

  def get_random_item_index(self):
    item_id = self.available_items_index[random.randint(0, self.sim_matrix.shape[0])]
    return item_id

  def get_n_closest(self, item_id, n=10):
    if item_id not in self.available_items_index:
      raise ValueError('Item is not available on data.')
    item_index = self.available_items_index.tolist().index(item_id)
    closest_n_iten = np.argsort(-self.sim_matrix[item_index])[1:n]
    n_closest = []
    for item_data in zip(self.available_items_index[closest_n_iten], self.sim_matrix[item_index][closest_n_iten]):
      n_closest.append(item_data)
    return tuple(n_closest)

Writing item_based_recommender.py


#Setup do Sistema de Recomendação

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [170]:
df_items = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Data Expert/Day 7/DNC_order_items_dataset.csv", index_col=0)

df_reviews = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Data Expert/Day 7/DNC_order_reviews_dataset.csv", index_col=0)

df_orders = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Data Expert/Day 7/DNC_orders_dataset.csv", index_col=0)

df_products = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Data Expert/Day 7/DNC_products_dataset.csv", index_col=0)

In [171]:
df1 = df_items.merge(df_products, how='inner', on='product_id')
df1.head()

,order_id,order_item_id,product_id,price,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,55.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,64.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0


In [172]:
df2 = df1.merge(df_reviews, how='inner', on='order_id')
df2.head()

,order_id,order_item_id,product_id,price,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_id,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,97ca439bc427b48bc1cd7177abe71365,5
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,55.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,b11cba360bbe71410c291b764753d37f,5
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,64.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,af01c4017c5ab46df6cc810e069e654a,4
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,8304ff37d8b16b57086fa283fe0c44f8,5
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,426f43a82185969503fb3c86241a9535,5


In [173]:
df = df2.merge(df_orders, how='inner', on='order_id')
df.head()

,order_id,order_item_id,product_id,price,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_id,review_score,customer_id,order_status
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,97ca439bc427b48bc1cd7177abe71365,5,3ce436f183e68e07877b285a838db11a,delivered
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,55.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,b11cba360bbe71410c291b764753d37f,5,e6eecc5a77de221464d1c4eaff0a9b64,delivered
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,64.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,af01c4017c5ab46df6cc810e069e654a,4,4ef55bf80f711b372afebcb7c715344a,delivered
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,8304ff37d8b16b57086fa283fe0c44f8,5,30407a72ad8b3f4df4d15369126b20c9,delivered
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,426f43a82185969503fb3c86241a9535,5,91a792fef70ecd8cc69d3c7feb3d12da,delivered


In [192]:
df.shape

(112372, 15)

In [168]:
#df.drop(['order_id', 'order_item_id', 'price', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'review_id', 'order_status'],axis=1,inplace=True)

In [ ]:
df.head()

,product_id,review_score,customer_id
0,4244733e06e7ecb4970a6e2683c13e61,5,3ce436f183e68e07877b285a838db11a
1,4244733e06e7ecb4970a6e2683c13e61,5,e6eecc5a77de221464d1c4eaff0a9b64
2,4244733e06e7ecb4970a6e2683c13e61,4,4ef55bf80f711b372afebcb7c715344a
3,4244733e06e7ecb4970a6e2683c13e61,5,30407a72ad8b3f4df4d15369126b20c9
4,4244733e06e7ecb4970a6e2683c13e61,5,91a792fef70ecd8cc69d3c7feb3d12da


In [ ]:
#df.drop(columns=['order_id', 'review_id', 'order_status'], inplace=True)

In [ ]:
df_top_produtos = df.groupby(by='product_id', as_index=False) \
    .agg({'customer_id': pd.Series.nunique}) \
    .rename(columns={'customer_id': 'customer_id_count'}) \
    .set_index('product_id') \
    .sort_values('customer_id_count', ascending = False)

In [ ]:
df_top_produtos.head()

,customer_id_count
product_id,
99a4788cb24856965c36a24e339b6058,458
aca2eb7d00ea1a7b8ebd4e68314663af,429
422879e10f46682990de24d770e7f83d,351
d1c427060a0f73f6b889a5c7c61f2ac4,320
389d119b48cf3043d311335e499d9c6b,310


In [ ]:
df_top_reviews = df.groupby(by='product_id', as_index=False) \
    .agg({'review_score': np.mean}) \
    .rename(columns={'review_score': 'review_score_avg'}) \
    .set_index('product_id') \
    .sort_values('review_score_avg', ascending = False)

In [ ]:
df_top_reviews.head()

,review_score_avg
product_id,
00066f42aeeb9f3007548bb9d3f33c38,5.0
60314cbf527af04ebc935be7c02dc1cd,5.0
5fe23e6307c231e287f13c0a4be06c88,5.0
5fe490e61e7a37f2f0d1ad1771ac027d,5.0
c696cd2db7870be08c37822f7c0ef831,5.0


In [ ]:
df.head()

,order_id,order_item_id,product_id,price,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_id,review_score,customer_id,order_status
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,97ca439bc427b48bc1cd7177abe71365,5,3ce436f183e68e07877b285a838db11a,delivered
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,55.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,b11cba360bbe71410c291b764753d37f,5,e6eecc5a77de221464d1c4eaff0a9b64,delivered
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,64.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,af01c4017c5ab46df6cc810e069e654a,4,4ef55bf80f711b372afebcb7c715344a,delivered
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,8304ff37d8b16b57086fa283fe0c44f8,5,30407a72ad8b3f4df4d15369126b20c9,delivered
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,426f43a82185969503fb3c86241a9535,5,91a792fef70ecd8cc69d3c7feb3d12da,delivered


In [174]:
df_drop = df.drop(columns=['order_id', 'order_item_id', 'review_id', 'customer_id', 'order_status'])

In [ ]:
#df_drop.drop(columns=['product_id'], inplace=True)

In [175]:
df_drop.shape

(112372, 10)

In [176]:
df_drop = df_drop.sample(1000)

In [177]:
df_product_id = df_drop['product_id']

In [178]:
df_drop.drop(columns=['product_id'], inplace=True)

In [179]:
df_drop_dummy = pd.get_dummies(df_drop)
df_drop_dummy.head()

,price,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_score,product_category_name_agro_industria_e_comercio,product_category_name_alimentos,product_category_name_alimentos_bebidas,product_category_name_artes_e_artesanato,product_category_name_audio,product_category_name_automotivo,product_category_name_bebes,product_category_name_bebidas,product_category_name_beleza_saude,product_category_name_brinquedos,product_category_name_cama_mesa_banho,product_category_name_casa_conforto_2,product_category_name_casa_construcao,product_category_name_climatizacao,product_category_name_consoles_games,product_category_name_construcao_ferramentas_construcao,product_category_name_construcao_ferramentas_ferramentas,product_category_name_construcao_ferramentas_iluminacao,product_category_name_construcao_ferramentas_jardim,product_category_name_construcao_ferramentas_seguranca,product_category_name_cool_stuff,product_category_name_eletrodomesticos,product_category_name_eletrodomesticos_2,product_category_name_eletronicos,product_category_name_eletroportateis,product_category_name_esporte_lazer,product_category_name_fashion_bolsas_e_acessorios,product_category_name_fashion_calcados,product_category_name_fashion_roupa_masculina,product_category_name_ferramentas_jardim,product_category_name_industria_comercio_e_negocios,product_category_name_informatica_acessorios,product_category_name_instrumentos_musicais,product_category_name_livros_interesse_geral,product_category_name_livros_tecnicos,product_category_name_malas_acessorios,product_category_name_market_place,product_category_name_moveis_cozinha_area_de_servico_jantar_e_jardim,product_category_name_moveis_decoracao,product_category_name_moveis_escritorio,product_category_name_moveis_quarto,product_category_name_moveis_sala,product_category_name_papelaria,product_category_name_pcs,product_category_name_perfumaria,product_category_name_pet_shop,product_category_name_relogios_presentes,product_category_name_telefonia,product_category_name_telefonia_fixa,product_category_name_utilidades_domesticas
16574,40.8,56.0,963.0,1100.0,16.0,15.0,15.0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
74266,334.8,54.0,446.0,277.0,23.0,15.0,23.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
86936,130.0,60.0,1014.0,150.0,30.0,10.0,15.0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43434,39.9,18.0,346.0,150.0,16.0,5.0,15.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
19285,149.9,58.0,2400.0,10800.0,54.0,27.0,36.0,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [180]:
df_drop_dummy = df_drop_dummy.fillna(0)

In [181]:
from sklearn.preprocessing import MinMaxScaler 

df_norm = MinMaxScaler()
df_norm1 = df_norm.fit_transform(df_drop_dummy.values)

In [182]:
matriz = np.zeros((1000,1000), dtype=np.float64) 

In [183]:
from scipy.spatial import distance 

for i in range(1000):
	for j in range(1000):
		matriz[i][j] = distance.euclidean(df_norm1[i], df_norm1[j]) 

In [184]:
print(matriz)

[[0.         1.42751402 1.44641907 ... 1.56614605 1.44559739 1.70879285]
 [1.42751402 0.         1.45316129 ... 1.56670946 1.45006975 1.70976198]
 [1.44641907 1.45316129 0.         ... 1.61666894 1.43062894 1.82177237]
 ...
 [1.56614605 1.56670946 1.61666894 ... 0.         1.66283152 0.51042406]
 [1.44559739 1.45006975 1.43062894 ... 1.66283152 0.         1.86264558]
 [1.70879285 1.70976198 1.82177237 ... 0.51042406 1.86264558 0.        ]]


In [185]:
df_matriz = pd.DataFrame(matriz)
df_matriz.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0.000000,1.427514,1.446419,1.720975,1.594407,1.478612,1.481384,1.472236,1.446880,1.522438,1.465143,1.608578,1.495493,1.444763,1.513153,1.448130,1.506768,1.574459,1.448975,1.435969,1.527723,1.444476,1.477249,1.588584,1.451651,1.513842,1.457200,1.467686,1.608464,1.553162,1.518171,1.472558,1.444104,1.443603,1.753202,1.502564,1.458735,1.611634,1.456686,1.475786,...,1.546695,1.470913,1.447288,1.473845,1.622753,1.491062,1.459018,1.449156,1.474408,1.493379,1.458496,1.499192,1.455532,1.458511,1.499645,1.419538,1.476840,1.446847,1.427995,1.506368,1.522738,1.451181,1.455689,1.839909,1.486955,1.424859,1.723909,1.618653,1.704162,1.524920,1.468013,1.535542,1.580290,1.470616,1.447785,1.456196,1.528279,1.566146,1.445597,1.708793
1,1.427514,0.000000,1.453161,1.708108,1.608360,1.452569,1.468668,1.482957,1.440633,0.518909,1.455626,1.605054,1.485080,0.303336,1.541347,1.442852,1.499123,1.573777,0.309372,1.423056,1.512900,0.306831,1.451960,1.558915,1.449207,1.482702,0.337926,1.463032,1.613928,1.524094,1.502298,1.454092,1.453082,0.343011,1.725233,1.527322,1.473433,1.610713,1.459776,1.455598,...,1.518522,1.473363,1.435582,1.461988,1.641251,1.454705,1.449864,1.447179,1.452887,1.480338,0.350731,1.492797,1.452863,1.452148,1.470674,1.425008,1.451440,1.425167,1.431266,1.518294,1.481286,1.443518,0.328201,1.804676,0.444925,1.443837,1.710817,1.606866,1.668501,1.561079,1.455314,1.585398,1.550298,1.502289,1.426884,1.445237,1.500979,1.566709,1.450070,1.709762
2,1.446419,1.453161,0.000000,1.870342,1.552005,1.444843,1.525450,1.485264,1.428867,1.528573,1.540258,1.743335,1.498062,1.519026,1.615532,1.432743,1.511323,1.636173,1.441240,1.473893,1.526214,1.434928,1.443102,1.586644,1.439564,1.467283,1.527725,1.432386,1.741641,1.523083,1.523767,1.458536,1.504101,1.428953,1.750463,1.479849,1.433600,1.743019,1.451904,1.452099,...,1.502936,1.453146,1.477739,1.462951,1.753965,1.485685,1.426461,1.437474,1.530175,1.491573,1.441404,1.541671,1.446803,1.454696,1.482907,1.452547,1.439531,1.459542,1.455354,1.609007,1.478080,1.507141,1.450363,1.893811,1.490687,1.448185,1.845524,1.747915,1.801596,1.508697,1.457511,1.522880,1.542008,1.456297,1.468424,1.498764,1.506743,1.616669,1.430629,1.821772
3,1.720975,1.708108,1.870342,0.000000,2.019115,1.878022,1.615765,1.809175,1.843867,1.739668,1.571795,1.527050,1.752033,1.598210,1.608617,1.848268,1.742683,1.637261,1.794700,1.679990,1.823554,1.816114,1.877472,1.911656,1.795269,1.896094,1.612518,1.827000,1.546231,1.933960,1.740271,1.834807,1.649421,1.828804,2.125112,1.938775,1.838540,1.534591,1.779106,1.793132,...,1.822520,1.824286,1.644816,1.766588,1.603926,1.781828,1.867803,1.801815,1.624686,1.754516,1.857347,1.736946,1.786519,1.785872,1.781222,1.722022,1.866247,1.725209,0.984121,1.588426,1.884190,1.638141,1.779340,1.774476,1.750168,1.743946,1.590855,1.580532,1.582530,1.905949,1.785757,1.935540,1.965045,1.911351,1.702452,1.633462,1.822846,1.757852,1.845703,1.701728
4,1.594407,1.608360,1.552005,2.019115,0.000000,0.695555,1.705415,1.476673,1.581413,1.710868,1.743635,1.892654,1.699815,1.698383,1.720365,1.612887,1.663406,1.699056,1.622613,1.639646,1.611967,1.627900,0.694942,1.543164,1.578463,0.565125,1.736121,1.552436,1.901546,1.511474,1.688875,1.546978,1.659322,1.613431,1.674919,1.489656,1.523829,1.838500,1.636524,0.701063,...,1.620334,1.517774,1.648254,1.633613,1.803326,0.741788,1.588354,1.626050,1.636854,1.635326,1.662184,1.566183,1.634101,1.650962,1.602957,1.631281,0.699249,1.613661,1.660755,1.771710,0.662873,0.848226,1.622292,1.942223,1.663203,1.544577,1.814530,1.844270,1.859491,1.473972,1.660089,1.548362,1.479320,1.485658,1.582298,1.669968,1.538064,1.566047,1.632057,1.794322


In [188]:
df_matriz.rename(columns=df_product_id, inplace=True)

In [189]:
df_matriz.head()

,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,436c8d57ff8d4aa254318e9bd9b48c83,4244733e06e7ecb4970a6e2683c13e61,e5f2d52b802189ee658865ca93d83a8f,c777355d18b72b67abbeef9df44fd0fd,c777355d18b72b67abbeef9df44fd0fd,928c2c0fdf465581daac95380af04b3e,c777355d18b72b67abbeef9df44fd0fd,7634da152a4610f1595efa32f14722fc,7634da152a4610f1595efa32f14722fc,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ef92defde845ab8450f9d70c526ef70f,ef92defde845ab8450f9d70c526ef70f,ef92defde845ab8450f9d70c526ef70f,ef92defde845ab8450f9d70c526ef70f,ef92defde845ab8450f9d70c526ef70f,8d4f2bb7e93e6710a28f34fa83ee7d28,8d4f2bb7e93e6710a28f34fa83ee7d28,8d4f2bb7e93e6710a28f34fa83ee7d28,557d850972a7d6f792fd18ae1400d9b6,310ae3c140ff94b03219ad0adc3c778f,310ae3c140ff94b03219ad0adc3c778f,...,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,a62e25e09e05e6faf31d90c6ec1aa3d1,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,389d119b48cf3043d311335e499d9c6b,53759a2ecddad2bb87a079a1f1519f73,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db
0,0.000000,1.427514,1.446419,1.720975,1.594407,1.478612,1.481384,1.472236,1.446880,1.522438,1.465143,1.608578,1.495493,1.444763,1.513153,1.448130,1.506768,1.574459,1.448975,1.435969,1.527723,1.444476,1.477249,1.588584,1.451651,1.513842,1.457200,1.467686,1.608464,1.553162,1.518171,1.472558,1.444104,1.443603,1.753202,1.502564,1.458735,1.611634,1.456686,1.475786,...,1.546695,1.470913,1.447288,1.473845,1.622753,1.491062,1.459018,1.449156,1.474408,1.493379,1.458496,1.499192,1.455532,1.458511,1.499645,1.419538,1.476840,1.446847,1.427995,1.506368,1.522738,1.451181,1.455689,1.839909,1.486955,1.424859,1.723909,1.618653,1.704162,1.524920,1.468013,1.535542,1.580290,1.470616,1.447785,1.456196,1.528279,1.566146,1.445597,1.708793
1,1.427514,0.000000,1.453161,1.708108,1.608360,1.452569,1.468668,1.482957,1.440633,0.518909,1.455626,1.605054,1.485080,0.303336,1.541347,1.442852,1.499123,1.573777,0.309372,1.423056,1.512900,0.306831,1.451960,1.558915,1.449207,1.482702,0.337926,1.463032,1.613928,1.524094,1.502298,1.454092,1.453082,0.343011,1.725233,1.527322,1.473433,1.610713,1.459776,1.455598,...,1.518522,1.473363,1.435582,1.461988,1.641251,1.454705,1.449864,1.447179,1.452887,1.480338,0.350731,1.492797,1.452863,1.452148,1.470674,1.425008,1.451440,1.425167,1.431266,1.518294,1.481286,1.443518,0.328201,1.804676,0.444925,1.443837,1.710817,1.606866,1.668501,1.

In [190]:
df_matriz.set_index(df_product_id, inplace=True)

In [191]:
df_matriz

,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,4244733e06e7ecb4970a6e2683c13e61,436c8d57ff8d4aa254318e9bd9b48c83,4244733e06e7ecb4970a6e2683c13e61,e5f2d52b802189ee658865ca93d83a8f,c777355d18b72b67abbeef9df44fd0fd,c777355d18b72b67abbeef9df44fd0fd,928c2c0fdf465581daac95380af04b3e,c777355d18b72b67abbeef9df44fd0fd,7634da152a4610f1595efa32f14722fc,7634da152a4610f1595efa32f14722fc,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ac6c3623068f30de03045865e4e10089,ef92defde845ab8450f9d70c526ef70f,ef92defde845ab8450f9d70c526ef70f,ef92defde845ab8450f9d70c526ef70f,ef92defde845ab8450f9d70c526ef70f,ef92defde845ab8450f9d70c526ef70f,8d4f2bb7e93e6710a28f34fa83ee7d28,8d4f2bb7e93e6710a28f34fa83ee7d28,8d4f2bb7e93e6710a28f34fa83ee7d28,557d850972a7d6f792fd18ae1400d9b6,310ae3c140ff94b03219ad0adc3c778f,310ae3c140ff94b03219ad0adc3c778f,...,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,a62e25e09e05e6faf31d90c6ec1aa3d1,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,389d119b48cf3043d311335e499d9c6b,53759a2ecddad2bb87a079a1f1519f73,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
84f5c4f480ad6c9998d6a6860f1a2e41,0.000000,1.427514,1.446419,1.720975,1.594407,1.478612,1.481384,1.472236,1.446880,1.522438,1.465143,1.608578,1.495493,1.444763,1.513153,1.448130,1.506768,1.574459,1.448975,1.435969,1.527723,1.444476,1.477249,1.588584,1.451651,1.513842,1.457200,1.467686,1.608464,1.553162,1.518171,1.472558,1.444104,1.443603,1.753202,1.502564,1.458735,1.611634,1.456686,1.475786,...,1.546695,1.470913,1.447288,1.473845,1.622753,1.491062,1.459018,1.449156,1.474408,1.493379,1.458496,1.499192,1.455532,1.458511,1.499645,1.419538,1.476840,1.446847,1.427995,1.506368,1.522738,1.451181,1.455689,1.839909,1.486955,1.424859,1.723909,1.618653,1.704162,1.524920,1.468013,1.535542,1.580290,1.470616,1.447785,1.456196,1.528279,1.566146,1.445597,1.708793
21767347dfdabbe42b3ca5846a4cdb48,1.427514,0.000000,1.453161,1.708108,1.608360,1.452569,1.468668,1.482957,1.440633,0.518909,1.455626,1.605054,1.485080,0.303336,1.541347,1.442852,1.499123,1.573777,0.309372,1.423056,1.512900,0.306831,1.451960,1.558915,1.449207,1.482702,0.337926,1.463032,1.613928,1.524094,1.502298,1.454092,1.453082,0.343011,1.725233,1.527322,1.473433,1.610713,1.459776,1.455598,...,1.518522,1.473363,1.435582,1.461988,1.641251,1.454705,1.449864,1.447179,1.452887,1.480338,0.350731,1.492797,1

In [213]:
top_csv = np.zeros((1,6),dtype=int)

for produto in range(0,df_matriz.shape[0]): # para cada cidade
  actual_produto = df_matriz.iloc[city]
  #print(cod_ibge)
  search = []
  for line in range(1,actual_city.shape[0]): # crio o dicionáário
      cod_produto = str(df_matriz.columns[line])
      dist = float(actual_produto[line])
      search.append((cod_produto, dist))

  search.sort(key=lambda x:x[1]) # ordeno pela distancia
  top5 = search[1:6] # pegar top5 cidades mais próximas
  regist = np.array([[ (actual_produto[0]),(top5[0][0]),(top5[1][0]),(top5[2][0]),(top5[3][0]),(top5[4][0]) ]] )
  top_csv = np.concatenate((top_csv,regist)) 

In [214]:
dataset = pd.DataFrame()
dataset['target_product'] = top_csv[:,0]
dataset['top1'] = top_csv[:,1]
dataset['top2'] = top_csv[:,2]
dataset['top3'] = top_csv[:,3]
dataset['top4'] = top_csv[:,4]
dataset['top5'] = top_csv[:,5]
dataset.drop(0,inplace=True) 

In [216]:
dataset.head()

,target_product,top1,top2,top3,top4,top5
product_id,,,,,,
84f5c4f480ad6c9998d6a6860f1a2e41,1.7087928528424237,99a4788cb24856965c36a24e339b6058,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,99a4788cb24856965c36a24e339b6058
21767347dfdabbe42b3ca5846a4cdb48,1.7087928528424237,99a4788cb24856965c36a24e339b6058,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,99a4788cb24856965c36a24e339b6058
b30d7b0adcdc4a117f455472548155ec,1.7087928528424237,99a4788cb24856965c36a24e339b6058,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,99a4788cb24856965c36a24e339b6058
ab330e1b64fde92d3b4d4d6980a38576,1.7087928528424237,99a4788cb24856965c36a24e339b6058,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,99a4788cb24856965c36a24e339b6058
25f86162fee18735fffdb762dcb10d7c,1.7087928528424237,99a4788cb24856965c36a24e339b6058,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,368c6c730842d78016ad823897a372db,99a4788cb24856965c36a24e339b6058
